In [4]:
import requests
import json
import pandas as pd
from time import sleep

err_count = 0
while err_count < 3:
    try:
        #小心requrest太過頻繁IP會被對方封鎖
        res1 = requests.get('http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20170901&stockNo=2330')
        sleep(5)
        res2 = requests.get('http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20171001&stockNo=2330')
        break
    except:
        sleep(5)
        err_count += 1
        continue
if err_count == 3:
    print('連線失敗')

json1 = res1.json()
json2 = res2.json()

In [5]:
header = json1['fields']
header

['日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數']

In [6]:
data = json1['data']
data.extend(json2['data'])
len(data)

41

In [7]:
df = pd.DataFrame.from_records(data, columns=header)
df.sort_index(ascending=False).head()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
40,106/10/31,"22,582,013","5,497,062,659",243.50,245.00,241.50,243.00,0.00,"8,176"
39,106/10/30,"30,241,990","7,358,619,170",242.00,245.00,241.50,243.00,+4.00,"10,609"
38,106/10/27,"18,040,541","4,304,460,160",237.50,240.00,237.50,239.00,+3.00,"5,715"
37,106/10/26,"12,959,717","3,069,618,429",237.00,238.00,236.00,236.00,-2.00,"4,947"
36,106/10/25,"17,205,187","4,099,505,436",238.50,239.00,237.50,238.00,0.00,"5,808"


In [8]:
#日期轉換西元年、如使用Jupyter重複執行這裡會讓日期溢位
for i, row in df.iterrows():
    #print(row[0])
    strDate = row[0]
    DateArr = strDate.split("/")
    DateArr[0] = str(int(DateArr[0])+1911)
    DateArr[1] = str(int(DateArr[1]))
    DateArr[2] = str(int(DateArr[2]))
    #df.set_value(i,'日期','/'.join(DateArr))
    df.loc[i, '日期'] = '/'.join(DateArr)
df.sort_index(ascending=False).head()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
40,2017/10/31,"22,582,013","5,497,062,659",243.50,245.00,241.50,243.00,0.00,"8,176"
39,2017/10/30,"30,241,990","7,358,619,170",242.00,245.00,241.50,243.00,+4.00,"10,609"
38,2017/10/27,"18,040,541","4,304,460,160",237.50,240.00,237.50,239.00,+3.00,"5,715"
37,2017/10/26,"12,959,717","3,069,618,429",237.00,238.00,236.00,236.00,-2.00,"4,947"
36,2017/10/25,"17,205,187","4,099,505,436",238.50,239.00,237.50,238.00,0.00,"5,808"


In [9]:
df.to_csv('../data/fp_demo.csv', sep=',', encoding='utf-8-sig', index=False)

In [10]:
import sqlite3 as sql

#連線至sqlite檔案，若無該檔案sql，則會建立一個新的
conn = sql.connect("../data/twse.db")
#將Dataframe資料寫入sql檔中的'demo1'表中，無該資料表則會自動建立
df.to_sql("demo1", conn, if_exists="replace", index=False)

In [11]:
pd.read_sql_query("select * from demo1;", conn).sort_index(ascending=False).head()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
40,2017/10/31,"22,582,013","5,497,062,659",243.50,245.00,241.50,243.00,0.00,"8,176"
39,2017/10/30,"30,241,990","7,358,619,170",242.00,245.00,241.50,243.00,+4.00,"10,609"
38,2017/10/27,"18,040,541","4,304,460,160",237.50,240.00,237.50,239.00,+3.00,"5,715"
37,2017/10/26,"12,959,717","3,069,618,429",237.00,238.00,236.00,236.00,-2.00,"4,947"
36,2017/10/25,"17,205,187","4,099,505,436",238.50,239.00,237.50,238.00,0.00,"5,808"
